# Autoencoder Built on Tensorflow

Load Tensorflow and MNIST

In [15]:
%matplotlib inline
from PIL import Image
import numpy as np
from numpy import *
from sklearn import preprocessing as prep
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import ListedColormap
from sklearn.decomposition import PCA, FactorAnalysis
import matplotlib.pyplot as plt
import sklearn.cross_validation as crval
from sklearn.naive_bayes import GaussianNB
from sklearn.datasets import fetch_mldata
from scipy.interpolate import interp1d
from sklearn.svm import LinearSVC
import pylab
from sklearn.svm import SVC
from sklearn.cluster import KMeans
from matplotlib import pyplot
from sklearn.mixture import GMM
from sklearn.neighbors import KernelDensity
from sklearn import datasets
from sklearn import linear_model
import matplotlib.pyplot as plt
from sklearn import svm
import csv
from mpl_toolkits.mplot3d import Axes3D
from scipy import stats
import warnings
warnings.filterwarnings("ignore")


In [16]:
import tensorflow as tf
#from tensorflow.examples.tutorials.mnist import input_data
#mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

In [3]:
#mnist.train.images[0:5]
X=[]
with open('density_data.csv','r') as f:
    reader = csv.reader(f)
    for row1 in reader:
        X.append(row1)
#print size(X)

X=np.asarray(X)
X=X[:,1:15] # removing the first column since it contains indexes

print 'Data dimensions: ' 
print X.shape

X=X.astype(np.float)


X= np.ravel(X)
X= X.reshape(5000, 14)
#print(shape(X))

# splitting .. 
selX = X # (atm) we take all data --> no splitting 

# scaling
std_scal=prep.StandardScaler()
selX_scaled=std_scal.fit_transform(selX)
print('Original mean: ' +str(mean(selX))+' and std:'+ str(std(selX)))
print('After-cleaning mean: '+str(mean(selX_scaled))+' and std: '+str(std(selX_scaled)))


X=selX_scaled
#for clusters in range(3,3, 3):
#kmeans = KMeans(n_clusters=20, random_state=0).fit(X)   

#cluster_labels= kmeans.score(X)



#print('done')
#print cluster_labels

Data dimensions: 
(5000, 14)
Original mean: 0.189320842874 and std:2.26790054052
After-cleaning mean: 4.3038588566e-17 and std: 1.0


Load numpy and PyPlot

In [4]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Print the first image for fun.

In [5]:
#img_0 = mnist.train.images[0]

In [6]:
w, h = 28, 28
#plt.imshow(img_0.reshape((w, h)), cmap='gray')

The autoencoder we are going to build is as below.

![](diagram.png)

With x and xp of size 28x28 and z of size 16x16.

Set the dimension of embedded layer.

In [7]:
wz, hz = 16, 16

Create a container for inputs.

In [8]:
x = tf.placeholder(tf.float32, shape=[None, 14])

Declare variables for model parameters.

In [9]:
W = tf.Variable(tf.truncated_normal([14,wz * hz], stddev=0.1))
b = tf.Variable(tf.truncated_normal([wz * hz], stddev=0.1))
Wp = tf.Variable(tf.truncated_normal([wz * hz,14], stddev=0.1))
bp = tf.Variable(tf.truncated_normal([14], stddev=0.1))

Set embedded and reconstruct formula.

In [10]:
z = tf.nn.relu(tf.matmul(x,W) + b)
xp = tf.nn.sigmoid(tf.matmul(z,Wp) + bp)

Define loss function, optimizer and training step.

In [11]:
loss = tf.reduce_mean(tf.square(x - xp))
optimizer = tf.train.RMSPropOptimizer(0.1)
train_step = optimizer.minimize(loss)

Create session and initialize variables.

In [12]:
sess = tf.Session()
init = tf.global_variables_initializer()
trained = False
sess.run(init)

[ERR] Load trained model if exists.

In [ ]:
#gives error
saver = tf.train.Saver()
ckpt = tf.train.get_checkpoint_state("./")
#tf.generate_checkpoint_state_proto()
if ckpt:
    print "loading: ", ckpt.model_checkpoint_path
    saver.restore(sess, ckpt.model_checkpoint_path)
    trained = True

In [13]:
xs = X
print xs.shape

(5000, 14)


Training.

In [17]:
for i in range(5000):
    if trained:
        print "Pre-trained model loaded - training skipped."
        break
    #xs, _ = mnist.train.next_batch(64)
    #print xs[0:10]
    sess.run(train_step, feed_dict={x: xs})
    if i % 1000 == 0:
        print 'i: ', i, 'loss:', sess.run(loss, feed_dict={x: xs})
    if i % 10000 == 0:
        saver.save(sess, './model.ckpt', global_step=i+1)

i:  0 loss: 1.24466


NameError: name 'saver' is not defined

In [ ]:
for i in range(5000):
    XP[i]=sess.run(xp, feed_dict={x: np.array([xs[i,:]])}).reshape((1, 14))
print XP

Plot some results (original image, embedded layer and reconstructed image).

In [ ]:
n = 3
for i in np.random.choice(range(len(mnist.train.images)), n):
    img = mnist.train.images[i]
    plt.figure(i)
    plt.subplot(131)
    plt.imshow(img.reshape((w, h)), cmap='gray')
    plt.subplot(132)
    plt.imshow(sess.run(z, feed_dict={x: np.array([img])}).reshape((wz, hz)), cmap='gray')
    plt.subplot(133)
    plt.imshow(sess.run(xp, feed_dict={x: np.array([img])}).reshape((w, h)), cmap='gray')

In [ ]:
# 3D visualization
trans_data=X_reduced_iso
X_iso=trans_data[0,:]
Y_iso=trans_data[1,:]
Z_iso=trans_data[2,:]

for angle in range(0, 180, 30):
    fig = pylab.figure()
    ax = Axes3D(fig)
    x=linspace(0,10,5000)

    ax = plt.gca()
    ax.scatter(X_iso, Y_iso, Z_iso, zorder=0.3)
    #ax.scatter(X_iso, Z_iso, zorder=0.3)
    
    ax.view_init(azim=angle)
    ax = plt.draw()
    plt.pause(.001)
